In [103]:
import csv
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
import seaborn as sns
sns.set()
sns.set_context("talk")
import re
import seaborn as sns
from IPython.display import display, Latex, Markdown
#from client.api.notebook import Notebook
#ok = Notebook('proj1.ok')

In [58]:
sent = pd.read_csv('data/vader_lexicon.txt', sep='\t', header=None)
sent = sent.drop([2, 3], 1)
sent.index.rename('token', inplace=True)
sent.rename(columns={1: 'polarity'}, inplace=True)
sent.head()
#print(len(sent))

,0,polarity
token,,
0,$:,-1.5
1,%),-0.4
2,%-),-1.5
3,&-:,-0.4
4,&:,-0.7


In [59]:
from datetime import datetime

ELEC_DATE = datetime(2016, 11, 8)
INAUG_DATE = datetime(2017, 1, 20)

In [60]:
trump_tweets = pd.read_csv('data/trump-tweet-in-office.csv')
trump_tweets.head()

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter for iPhone,RT @RepLeeZeldin: This impeachment inquiry has...,10-25-2019 02:10:02,8819,0,True,1187551862616141824
1,Twitter for iPhone,RT @RepDougCollins: Democrats have no concern ...,10-25-2019 02:08:26,13713,0,True,1187551461808492544
2,Twitter for iPhone,RT @RepGosar: The only reason Adam Schiff is c...,10-25-2019 02:08:24,10961,0,True,1187551451775750144
3,Twitter for iPhone,RT @RepArrington: Let's call this sham what it...,10-25-2019 02:07:49,4547,0,True,1187551305281212416
4,Twitter for iPhone,RT @RepMattGaetz: Democrats are trying to use ...,10-25-2019 02:07:37,10491,0,True,1187551256371515392


In [61]:
trump = trump_tweets.copy()
trump['id']=trump['id_str']
trump = trump.set_index(['id'])
trump.head()

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
id,,,,,,,
1187551862616141824,Twitter for iPhone,RT @RepLeeZeldin: This impeachment inquiry has...,10-25-2019 02:10:02,8819,0,True,1187551862616141824
1187551461808492544,Twitter for iPhone,RT @RepDougCollins: Democrats have no concern ...,10-25-2019 02:08:26,13713,0,True,1187551461808492544
1187551451775750144,Twitter for iPhone,RT @RepGosar: The only reason Adam Schiff is c...,10-25-2019 02:08:24,10961,0,True,1187551451775750144
1187551305281212416,Twitter for iPhone,RT @RepArrington: Let's call this sham what it...,10-25-2019 02:07:49,4547,0,True,1187551305281212416
1187551256371515392,Twitter for iPhone,RT @RepMattGaetz: Democrats are trying to use ...,10-25-2019 02:07:37,10491,0,True,1187551256371515392


In [62]:
repl_punc = trump['text'].str.replace(r"[?|,|$|!|&|.|#|^|;|:|/|\"]", ' ')
replacequotes = repl_punc.str.replace(r'^"', '')
#replacequotes = repl_punc.strip('"')
#replacequotes.loc['907675638055743489'].strip('\'"') 
#repl_punc.loc['1187551862616141824']

In [63]:
# Save your regex in punct_re
punct_re = r"[^\w\s]"
#[?|,|$|!|&|.|#|^|;|:|/|\"]
trump['no_punc'] = trump['text'].str.replace(punct_re, " ")

In [64]:
trumpcopy = trump.copy()
changed_punc = trumpcopy['no_punc'].str.replace('\s+', ' ')
trumpcopy['no_punc'] = changed_punc
splt = trumpcopy['no_punc'].str.split(expand=True).stack().reset_index().set_index('id')
splt.columns = ['num', 'word']


In [65]:
splt.head(10)


,num,word
id,,
1187551862616141824,0,RT
1187551862616141824,1,RepLeeZeldin
1187551862616141824,2,This
1187551862616141824,3,impeachment
1187551862616141824,4,inquiry
1187551862616141824,5,has
1187551862616141824,6,produced
1187551862616141824,7,NOTHING
1187551862616141824,8,to


In [78]:
tidy_format = splt
newt = tidy_format.reset_index()
newsent = sent.reset_index().rename(columns = {'token': 'word'})

In [79]:
display(newt.head())
display(newsent.head())
#display(newt.describe())
#display(newsent.describe())


,id,num,word
0,1187551862616141824,0,RT
1,1187551862616141824,1,RepLeeZeldin
2,1187551862616141824,2,This
3,1187551862616141824,3,impeachment
4,1187551862616141824,4,inquiry


,word,0,polarity
0,0,$:,-1.5
1,1,%),-0.4
2,2,%-),-1.5
3,3,&-:,-0.4
4,4,&:,-0.7


In [86]:
newt = tidy_format.reset_index()
#newt = newt.drop('num')
newt1 = newt[['id', 'word']].copy()
newsent = sent.reset_index().rename(columns = {0: 'word'})
newsent1 =newsent[['polarity', 'word']].copy()
display(newt1.head())
display(newsent1.head(300))

,id,word
0,1187551862616141824,RT
1,1187551862616141824,RepLeeZeldin
2,1187551862616141824,This
3,1187551862616141824,impeachment
4,1187551862616141824,inquiry


,polarity,word
0,-1.5,$:
1,-0.4,%)
2,-1.5,%-)
3,-0.4,&-:
4,-0.7,&:
...,...,...
295,2.0,l
296,2.2,l&r
297,1.3,laoj
298,2.0,lmao


In [87]:
polar = pd.merge(newt1, newsent1, how='outer', on= 'word')
newpol = polar.groupby('id').sum()
###newpol = newpol.drop('num', 1).fillna(0)
newpol = newpol.fillna(0)

In [89]:
polarities = newpol
maybpol = polarities.reset_index()
maybtrump = trump.reset_index()

In [91]:
display(maybpol.head())
display(maybtrump.head())

,id,polarity
0,8.222424e+17,1.7
1,8.222987e+17,2.7
2,8.224214e+17,-1.0
3,8.225018e+17,1.7
4,8.225019e+17,1.4


,id,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str,no_punc
0,1187551862616141824,Twitter for iPhone,RT @RepLeeZeldin: This impeachment inquiry has...,10-25-2019 02:10:02,8819,0,True,1187551862616141824,RT RepLeeZeldin This impeachment inquiry has...
1,1187551461808492544,Twitter for iPhone,RT @RepDougCollins: Democrats have no concern ...,10-25-2019 02:08:26,13713,0,True,1187551461808492544,RT RepDougCollins Democrats have no concern ...
2,1187551451775750144,Twitter for iPhone,RT @RepGosar: The only reason Adam Schiff is c...,10-25-2019 02:08:24,10961,0,True,1187551451775750144,RT RepGosar The only reason Adam Schiff is c...
3,1187551305281212416,Twitter for iPhone,RT @RepArrington: Let's call this sham what it...,10-25-2019 02:07:49,4547,0,True,1187551305281212416,RT RepArrington Let s call this sham what it...
4,1187551256371515392,Twitter for iPhone,RT @RepMattGaetz: Democrats are trying to use ...,10-25-2019 02:07:37,10491,0,True,1187551256371515392,RT RepMattGaetz Democrats are trying to use ...


In [93]:
senti = pd.merge(maybpol, maybtrump, how='outer', on='id').set_index('id')


11568 16661


,polarity,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str,no_punc
id,,,,,,,,,
8.222424e+17,1.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8.222987e+17,2.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8.224214e+17,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8.225018e+17,1.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8.225019e+17,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
new_senti= senti.loc[senti['favorite_count']>=0].drop('id_str',axis=1)
print(len(maybtrump),len(new_senti))
display(new_senti.head())

11568 11568


,polarity,source,text,created_at,retweet_count,favorite_count,is_retweet,no_punc
id,,,,,,,,
8.225021e+17,3.7,Twitter for iPhone,What truly matters is not which party controls...,01-20-2017 17:52:45,42790.0,180394.0,False,What truly matters is not which party controls...
8.225026e+17,3.9,Twitter for iPhone,We will bring back our jobs. We will bring bac...,01-20-2017 17:54:36,37458.0,173821.0,False,We will bring back our jobs We will bring bac...
8.225029e+17,0.0,Twitter for iPhone,We will follow two simple rules: BUY AMERICAN ...,01-20-2017 17:55:44,38092.0,161358.0,False,We will follow two simple rules BUY AMERICAN ...
8.225041e+17,0.0,Twitter for iPhone,So to all Americans in every city near and far...,01-20-2017 18:00:43,14039.0,86467.0,False,So to all Americans in every city near and far...
8.226691e+17,2.7,Twitter for iPhone,THANK YOU for another wonderful evening in Was...,01-21-2017 04:56:15,52468.0,289942.0,False,THANK YOU for another wonderful evening in Was...


In [131]:
keytags_politics_partisans =['hilary','wall','immigration','Nancy Pelsi','dem\W','dem party','impeach'
                             ,'fake news']
keytags_foreign_policys =['Nato\W','israel','israel','Saudi\W','iran\W','iraq\W','middel east','Russia'
                          ,'japan\W','korean','kim ju']
keytags_trade_war =['tpp','currency manipulate|currency manipulation','wto','unfair[\w\W]+trade'
                    ,'trade deficits|trade deficit','trade deal|trade deals','china[\w\W]+trade','trade','china','tariff'
                    ,'american[\w\W]+manufac','american first']
keytags_trade_war_senti =['worst trade deal','currency manipulate|currency manipulation','china[\w\W]+tariff','new tariff','unfair[\w\W]+trade','taken advantage']


In [132]:
trade_tags_all = []
sentiments_tags_all = []
trade_tags_score = []

df_tweets = new_senti.copy()
counter =0


In [133]:

#do my own trade key tag point calculation
# regular trade key tag +1
# senti key tag +3

for i in range(len(df_tweets)) :
    #print(i)
    tagscore =0 
    senti_tag =[]
    trade_tag =[]

    tweet_text = df_tweets.iloc[i,2]
    for tag in keytags_trade_war:
        trade_tag = trade_tag + re.findall(tag,tweet_text)
    
    sentiments_tag =[]
    for tag in keytags_trade_war_senti:
        senti_tag = senti_tag + re.findall(tag,tweet_text)

    trade_tags_all.append(trade_tag)
    sentiments_tags_all.append(senti_tag)
    
    if len(trade_tag)>0:
        counter +=1
        tagscore += 1
        print(i, trade_tag)
    if len(senti_tag)>0:
        counter +=1
        tagscore += 3
        print(i, trade_tag)
        #print(tweet_text)
    trade_tags_score.append(tagscore)   
        
print(counter)

20 ['trade deficit', 'trade']
24 ['trade deficits', 'trade']
24 ['trade deficits', 'trade']
190 ['trade deficits', 'trade']
197 ['trade']
341 ['trade']
342 ['trade']
352 ['trade deficit', 'trade']
501 ['trade deal', 'trade']
538 ['trade deal', 'trade']
574 ['trade deal', 'trade']
649 ['trade']
729 ['trade deal', 'trade']
729 ['trade deal', 'trade']
731 ['trade deal', 'trade']
732 ['trade']
813 ['trade deal', 'trade']
813 ['trade deal', 'trade']
937 ['trade']
1088 ['trade']
1241 ['trade']
1242 ['trade']
1543 ['trade']
1687 ['unfair trade', 'trade']
1687 ['unfair trade', 'trade']
1696 ['trade', 'tariff']
1697 ['trade deficits', 'trade']
1714 ['trade', 'tariff']
1717 ['trade', 'tariff', 'tariff']
1730 ['unfair trade', 'trade']
1730 ['unfair trade', 'trade']
1805 ['unfair trade', 'trade']
1805 ['unfair trade', 'trade']
1815 ['trade']
1834 ['trade']
1911 ['unfairly for many years by many countries on trade', 'trade']
1911 ['unfairly for many years by many countries on trade', 'trade']
1937 

In [136]:
df_tweets_padded = df_tweets.copy()
df_tweets_padded['trade_tags_score']=trade_tags_score
df_tweets_padded['tradetag']=trade_tags_all
df_tweets_padded['senti_tradetag']=sentiments_tags_all

#df_tweets_padded.head()


In [137]:
df_tweets_trade_flagged = df_tweets_padded.loc[df_tweets_padded['tradetag'].str.len()>0 ]
#df_tweets_trade_flagged.head()
#df_tweets_trade_flagged['created_at'] =  pd.to_datetime(df_tweets_trade_flagged['created_at']).dt.date
df_tweets_trade_flagged.head(20)


,polarity,source,text,created_at,retweet_count,favorite_count,is_retweet,no_punc,trade_tags_score,tradetag,senti_tradetag
id,,,,,,,,,,,
8.246158e+17,-1.7,Twitter for Android,The U.S. has a 60 billion dollar trade deficit...,01-26-2017 13:51:46,25881.0,106893.0,False,The U S has a 60 billion dollar trade deficit...,1,"[trade deficit, trade]",[]
8.249700e+17,0.8,Twitter for Android,Mexico has taken advantage of the U.S. for lon...,01-27-2017 13:19:10,30604.0,160660.0,False,Mexico has taken advantage of the U S for lon...,4,"[trade deficits, trade]",[taken advantage]
8.475732e+17,-2.7,Twitter for iPhone,The meeting next week with China will be a ver...,03-30-2017 22:16:17,12126.0,58102.0,False,The meeting next week with China will be a ver...,1,"[trade deficits, trade]",[]
8.481971e+17,0.0,Twitter for iPhone,RT @DanScavino: .@POTUS @realDonaldTrump signs...,04-01-2017 15:35:12,7035.0,0.0,True,RT DanScavino POTUS realDonaldTrump signs...,1,[trade],[]
8.685012e+17,3.1,Twitter for iPhone,Just left the #G7Summit. Had great meetings on...,05-27-2017 16:16:47,8056.0,46903.0,False,Just left the G7Summit Had great meetings on...,1,[trade],[]
8.685014e+17,1.6,Twitter for iPhone,"....""we push for the removal of all trade-dist...",05-27-2017 16:17:30,7471.0,41671.0,False,we push for the removal of all trade dist...,1,[trade],[]
8.695038e+17,-4.6,Twitter for iPhone,We have a MASSIVE trade deficit with Germany p...,05-30-2017 10:40:36,22631.0,89874.0,False,We have a MASSIVE trade deficit with Germany p...,1,"[trade deficit, trade]",[]
8.806028e+17,1.9,Twitter for iPhone,Just finished a very good meeting with the Pre...,06-30-2017 01:44:01,19755.0,92733.0,False,Just finished a very good meeting with the Pre...,1,"[trade deal, trade]",[]
8.840082e+17,3.3,Twitter for iPhone,The G 20 Summit was a great success for the U....,07-09-2017 11:15:58,17005.0,81459.0,False,The G 20 Summit was a great success for the U ...,1,"[trade deal, trade]",[]
